Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Load Iris Dataset

In [2]:
# Load dataset
iris = load_iris()
X = iris.data        # Features (150, 4)
y = iris.target      # Labels (0,1,2)

print(X.shape, y.shape)

(150, 4) (150,)


Train-Test Split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Feature Scaling

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Convert Data to PyTorch Tensors

In [5]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

Define Neural Network Model

In [6]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)   # Input → Hidden
        self.relu = nn.ReLU()                           # ReLU activation
        self.fc2 = nn.Linear(hidden_size, output_size)  # Hidden → Output

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

Initialize Model, Loss, Optimizer

In [7]:
input_size = 4     # Iris has 4 features
hidden_size = 16
output_size = 3    # 3 classes

model = MLP(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()   # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

Train the Model

In [8]:
num_epochs = 100

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.8204
Epoch [20/100], Loss: 0.4987
Epoch [30/100], Loss: 0.3389
Epoch [40/100], Loss: 0.2591
Epoch [50/100], Loss: 0.1971
Epoch [60/100], Loss: 0.1479
Epoch [70/100], Loss: 0.1149
Epoch [80/100], Loss: 0.0924
Epoch [90/100], Loss: 0.0781
Epoch [100/100], Loss: 0.0688


Evaluate Model on Test Data

In [9]:
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)

    correct = (predicted == y_test_tensor).sum().item()
    total = y_test_tensor.size(0)

    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 93.33%
